In [5]:
import findspark
findspark.init()
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
import emojis

In [6]:
from pyspark.sql import SparkSession

# Инициализируем SparkSession
spark = SparkSession.builder \
    .master('local') \
    .appName('Dreamlone') \
    .config('spark.executor.memory', '4gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

# Task 4

### find emoticons in posts and post comments (negative, positive, neutral) (you can use external libraries or predefined emoticon lists) (use the spark udf and broadcast function for emoticons)

In [7]:
from pyspark.sql.types import StringType, BooleanType
from pyspark import *

# Загружаем файл, в котором будем все искать
itmo_posts = spark.read.parquet("./bigdata20/posts_api.parquet")
itmo_posts.createOrReplaceTempView("posts")

# Воспользуемся внешей библиотекой
def extract_emoji(text):
    return(emojis.get(text))
spark.udf.register("extract_emoji", extract_emoji)

data_emjs = spark.sql("SELECT key, extract_emoji(text) as emoji FROM posts WHERE text is not null")
data_emjs.select('key', 'emoji') \
.sort(desc("emoji")) \
.coalesce(1).write.format("json").save("./bigdata20/Answers/task4_emojies.json")